In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
X_full = pd.read_csv("/kaggle/input/mobile-price-classification/train.csv")
X_full.head()

In [ ]:
X_full.info()

In [ ]:
X_test_full = pd.read_csv('/kaggle/input/mobile-price-classification/test.csv', index_col='id')
X_test_full.head()

In [ ]:
X_test_full.info()

In [ ]:
from sklearn.model_selection import train_test_split

# Remove rows with missing target, separate target from predictors
y = X_full.price_range
X_full.drop(['price_range'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [ ]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

In [ ]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [ ]:
X_train.head()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy = 'median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
randomForestClassifier = RandomForestClassifier(n_estimators=100, random_state=0) 
kNeighborsClassifier = KNeighborsClassifier(n_neighbors=4)
supportVectorMachines = SVC(C=1,kernel='linear',gamma = 0.001)
models = [randomForestClassifier, kNeighborsClassifier, supportVectorMachines]

In [ ]:
score = -9999
for model in models:

    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('model', model)
                                 ])

    # Preprocessing of training data, fit model 
    my_pipeline.fit(X_train, y_train)

    # Preprocessing of validation data, get predictions
    preds = my_pipeline.predict(X_valid)

    print("============", model ,"=================")
    # Evaluate the model
    score = mean_absolute_error(y_valid, preds)
    print('MAE:', score)
    accuracy = accuracy_score(y_valid, preds)
    print('Accuracy:', accuracy)
    if accuracy > score:
        score = accuracy
        best_pipeline = my_pipeline
print('\n\n\nBest Pipeline:', my_pipeline)

In [ ]:
preds_test = my_pipeline.predict(X_test)

In [ ]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

Please give an upvote if you like it and feel free to ask any questions in comments. Thank you :)